# 1. Autoregressive Models
a. Write out the chain rule for a distribution over $x_1, x_2,...,x_n$.

$$ p(x_1) * p(x_2 | x_1) * ... * p(x_n | x_{n-1}, x_{n-2},...,x_1) $$

b. Draw an example of a Masked Autoregressive Distribution Estimation (MADE) model and illustrate the key characteristic that qualify it as proper probability model.

# 2. KL-Divergence
Let $\mu:\chi\rightarrow\mathbb{R}^k, \sigma, \chi\rightarrow\mathbb{R}^k$ and $q(z|x)=N(z;\mu(x),\textnormal{diag}(\sigma^2(x)))$. Suppose that $p(z)=N(z;0,I)$, show that

$$ D(q(z|x) || p(z)) = \frac{1}{2}\sum_i \sigma_i^2(x)+\mu(x)_i^2-log\sigma_i^2(x)-1 $$

# 3. Normalizing Flows
Let $q_0$ be a probability distribution on $z$, and define $\mathbf{z}_s=g_s(\mathbf{z}_{t-1})$ where $g_s:z\rightarrow z$ are invertible functions. Prove that the pushforward distribution on $\mathbf{z}_t=g_t \circ ... \circ g_1(\mathbf{z}_0)$ is given by $q_t$, where
$$ \log q_t(\mathbf{z}_t)=log q_0(z_0)-\sum_{s=1}^t\log\det\frac{\partial g_s(\mathbf{z}_{s-1})}{\partial\mathbf{z}_{s-1}} $$
Hint: consider using the inverse function theorem.

# 

In [3]:
import numpy as np